In [1]:
import urllib.request
import json
import pandas as pd
bechdel_data = pd.read_csv('bechdel.csv',encoding='utf-8')
bechdel_data = bechdel_data.set_index('imdb_id')
bechdel_data.sample(3)        
testurl = 'http://www.omdbapi.com/?i=tt0848228&apikey=949d45c1&plot=full'  # The Avengers (2012)
page = urllib.request.urlopen(testurl).read().decode('utf-8')
x = json.loads(page,encoding='utf-8')
x

{'Actors': 'Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth',
 'Awards': 'Nominated for 1 Oscar. Another 37 wins & 79 nominations.',
 'BoxOffice': '$623,279,547',
 'Country': 'USA',
 'DVD': '25 Sep 2012',
 'Director': 'Joss Whedon',
 'Genre': 'Action, Sci-Fi',
 'Language': 'English, Russian, Hindi',
 'Metascore': '69',
 'Plot': "Nick Fury is the director of S.H.I.E.L.D., an international peace-keeping agency. The agency is a who's who of Marvel Super Heroes, with Iron Man, The Incredible Hulk, Thor, Captain America, Hawkeye and Black Widow. When global security is threatened by Loki and his cohorts, Nick Fury and his team will need all their powers to save the world from disaster.",
 'Poster': 'https://images-na.ssl-images-amazon.com/images/M/MV5BMTk2NTI1MTU4N15BMl5BanBnXkFtZTcwODg0OTY0Nw@@._V1_SX300.jpg',
 'Production': 'Walt Disney Pictures',
 'Rated': 'PG-13',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.1/10'},
  {'Source': 'Rotten Tomatoes', 'Value'

In [12]:
omdb_data = dict()
exceptions = list()
id_list = bechdel_data.index.tolist()
#id_list
totalmovies = len(id_list)
print(totalmovies)
i = 0
for theid in id_list:
    i = i + 1
    id_string = 'tt'    
    needed_zeroes = 7 - len(str(theid))
    for j in range(0,needed_zeroes):
        id_string = id_string + '0'
    id_string = id_string + str(theid)
    omdburl = 'http://www.omdbapi.com/?i={0}&apikey=949d45c1'.format(id_string)
    try:
        omdbpage = urllib.request.urlopen(omdburl).read().decode('utf-8')
        omdb_data[theid] = json.loads(omdbpage,encoding='utf-8')
    except:
        exceptions.append(theid)
        pass
    if (i % 1000 == 0):
        print('Processing movie {0:,.0f} of {1:,.0f}...'.format(i,totalmovies))
print(exceptions)
for theid in exceptions:
    print()


7319
Processing movie 1,000 of 7,319...
Processing movie 2,000 of 7,319...
Processing movie 3,000 of 7,319...
Processing movie 4,000 of 7,319...
Processing movie 5,000 of 7,319...
Processing movie 6,000 of 7,319...
Processing movie 7,000 of 7,319...
[84516, 1019452]




In [13]:
#from pprint import pprint
ratings_sources = []
for thismovie in omdb_data:   # iterate through each movie by its key (imdb_id)
    theseratings = omdb_data[thismovie].get('Ratings')  # save the Ratings dict of this movie, if it exists
    if (theseratings is not None):           # if ratings exist for this movie...
        for i in range(0,len(theseratings)):   # iterate through each rating, each of which is a dict
            for thiskey in theseratings[i]:    # iterate through the keys for each rating in this one movie
                if (thiskey == 'Source'):      # if the key is 'Source', this might be a new source of ratings
                    if (theseratings[i]['Source'] not in ratings_sources):  # is the key already in the list of sources?
                        ratings_sources.append(theseratings[i]['Source'])         # if not, add it to the list of sources.
print(ratings_sources)
# w00t, so the only sources of ratings are IMDb, Rotten Tomatoes, and Metacritic! And the only one we're missing is RT.

for thismovie in omdb_data:   # iterate through each movie by its key (imdb_id)
    theseratings = omdb_data[thismovie].get('Ratings')  # save the Ratings dict of this movie, if it exists
    if (theseratings is not None):           # if ratings exist for this movie...
        for i in range(0,len(theseratings)):   # iterate through each rating, each of which is a dict
            for thiskey in theseratings[i]:    # iterate through the keys for each rating in this one movie
                if (theseratings[i][thiskey] == 'Rotten Tomatoes'):
                    omdb_data[thismovie]['RottenTomatoesScore'] = theseratings[i]['Value']
print('Done')

['Internet Movie Database', 'Rotten Tomatoes', 'Metacritic']
Done


In [35]:
omdb_df = pd.DataFrame.from_dict(omdb_data,orient='index')
omdb_df.index.name = 'imdb_id'

#omdb_df.sample(3).transpose()
#omdb_df[omdb_df.index == 1409024].transpose()   # Star Wars: The Force Awakens
omdb_df.to_csv('omdb.csv',encoding='utf-8')
print('Done')

Done


In [37]:
import numpy as np
#import pandas as pd
import time
omdb_df = pd.read_csv('omdb.csv',encoding='utf-8') 

# Drop non-movie-related columns
omdb_df.drop(['Season','seriesID','Episode','totalSeasons'],axis=1,inplace=True)

# Rename columns to what we want them to be.
omdb_df=omdb_df.rename(columns = {'Actors':'actors','Writer':'writers','Response':'response','Language':'languages'})
omdb_df=omdb_df.rename(columns = {'Title':'imdbtitle','Rated':'mpaa_rating','imdbRating':'userscore','Type':'type'})
omdb_df=omdb_df.rename(columns = {'Poster':'posterlink','BoxOffice':'imdb_boxoffice','Country':'countries'})
omdb_df=omdb_df.rename(columns = {'Ratings':'all_ratings_list','Genre':'genres','Runtime':'runtime','Metascore':'metascore'})
omdb_df=omdb_df.rename(columns = {'Awards':'awards','Plot':'imdb_plot','DVD':'dvd_release_date','Director':'director'})
omdb_df=omdb_df.rename(columns = {'Website':'website','Year':'imdbyear','Released':'imdb_release_date','imdbID':'imdb_id_str'})
omdb_df=omdb_df.rename(columns = {'Production':'studio','imdbVotes':'nVotes'})

# Drop non-movies
omdb_df = omdb_df[omdb_df['type'] == 'movie']

# Fix runtimes to be integers
omdb_df['runtime'] = omdb_df['runtime'].str.slice(stop=-4).replace('',np.nan)
omdb_df['runtime'] = omdb_df['runtime'].apply(float)
omdb_df['runtime'] = omdb_df['runtime'].apply(lambda x: '{:.0f}'.format(x))



# Fix release dates to be datetime, formatted as yyyy-mm-dd
omdb_df['imdb_release_date'] = pd.to_datetime(omdb_df['imdb_release_date'],format="%d %b %Y", errors='coerce')

# Convert Release dateimte into new columns for year, month, and week
#omdb_df['imdb_release_year'] = omdb_df['imdb_release_date'].dt.strftime('%Y')
#omdb_df['imdb_release_month'] = omdb_df['imdb_release_date'].dt.strftime('%B')
#omdb_df['imdb_release_weekday'] = omdb_df['imdb_release_date'].dt.strftime('%A')

#omdb_df['imdb_release_week'] = omdb_df['imdb_release_date'].apply(lambda x:x.week)

# Take the imdbVotes formatted as string containing "N/A" and comma-delimited thousands, convert to float
omdb_df['nVotes'] = omdb_df['nVotes'].dropna().replace('N/A',np.nan).dropna().apply(lambda x:float(x.replace(',','')))

#imdb_df[['imdb_id','imdbtitle','mpaa_rating','type','imdb_release_date','imdb_release_year','imdb_release_month','imdb_release_week','imdb_release_weekday']]
omdb_df['nVotes'] = omdb_df['nVotes'].apply(lambda x: '{:,.0f}'.format(x))

# Take the Metascore formatted as string containing "N/A", convert to float
# Also divide by 10 to make effect sizes more comparable
omdb_df['metascore'] = omdb_df['metascore'].dropna().replace('N/A',np.nan).dropna().apply(float)

# Take the imdbRating formatted as string containing "N/A", convert to float
omdb_df['userscore'] = omdb_df['userscore'].dropna().replace('N/A',np.nan).dropna().apply(float)

# Remove % from rotten tomatoes scores, convert to float
omdb_df['RottenTomatoesScore'] = omdb_df['RottenTomatoesScore'].dropna().apply(lambda x: str(x)[0:-1]).apply(float)

# Create indicator variables for English and USA movies (English is at least one of its languages etc.) (0 = no, 1 =yes)
omdb_df['english'] = (omdb_df['languages'].dropna().str.contains('nglish')).astype(int)
omdb_df['usa'] = (omdb_df['countries'].dropna().str.contains('USA')).astype(int)

# Convert imdb_ID to int, set it as the index
omdb_df['imdb_id'] = omdb_df['imdb_id'].dropna().apply(int)
omdb_df = omdb_df.set_index('imdb_id')

#omdb_df.sample(3).transpose()
#omdb_df[omdb_df.index == 1409024].transpose()   # Star Wars: The Force Awakens
omdb_df.to_csv('omdb.csv',encoding='utf-8')
print('Done')

Done
